In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm
from rich.console import Console
import time
from scrap import Request

In [2]:
# from geopy.geocoders import Nominatim
# geolocalização
# geolocator = Nominatim(user_agent="myApp", timeout=5)
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MyApp", timeout=5)
console = Console()
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)

In [3]:
def remove_bairro(address):
    if address.count('-') > 1:
        try:
            l_address = address.split('-')
            l_address[1] = l_address[1].split(',')[1]
        except Exception as e:
            print(address)
            return address
        return '-'.join(l_address)
    return address

In [4]:
with open('data/houses.pkl', 'rb') as f:
    obj = pickle.load(f)

In [5]:
df = pd.DataFrame(obj.values())

Ajustando o valor do imóvel

In [6]:
df = pd.concat([df, pd.DataFrame(df.price.to_dict().values(), columns=['rent', 'Condomínio', 'Preço c/ condomínio','IPTU'])], axis=1)

Ajustando o banheiro, removendo a lista

In [7]:
df.bathrooms = df.bathrooms.map(lambda x: int(x[0]) if isinstance(x, list) and (not not x) else 0)

In [8]:
df.iloc[0:3].address.apply(geolocator.geocode).apply(lambda x: pd.Series([x.latitude, x.longitude], index=['location_lat', 'location_long']))

,location_lat,location_long
0,-23.651039,-46.719276
1,-23.723508,-46.693460
2,-21.132831,-51.102942


In [9]:
df

,type,address,url,price,total_price,area,bedrooms,bathrooms,parking,amenities,description,transport,publisher_name,code,title,rent,Condomínio,Preço c/ condomínio,IPTU
0,Aluguel,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 3900, 'Condomínio': ' R$ 746 ', 'Preç...",None,80.0,2.0,21,1.0,"[Piscina, Churrasqueira, Condomínio fechado, A...",None,None,None,None,Apartamento com 2 Quartos e 2 banheiros para...,3900.0,R$ 746,R$ 4.646,R$ 225
1,Aluguel,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,"{'rent': 900, 'Condomínio': ' Não informado ',...",None,40.0,1.0,1,NaN,[],None,None,None,None,Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,Não informado,NaN,Não informado
2,Aluguel,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 2500, 'Condomínio': ' R$ 726 ', 'Preç...",None,38.0,1.0,1,1.0,"[Churrasqueira, Academia, Piscina]",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,R$ 726,R$ 3.226,NaN
3,Não informado,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 1700, 'Condomínio': ' R$ 350 ', 'Preç...",None,27.0,1.0,11,NaN,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,R$ 350,R$ 2.050,NaN
4,Aluguel,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 3500, 'Condomínio': ' R$ 706 ', 'Preç...",None,45.0,1.0,11,1.0,"[Piscina, Churrasqueira, Elevador, Condomínio ...",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,R$ 706,R$ 4.206,R$ 296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Aluguel,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 3700, 'Condomínio': ' R$ 950 ', 'Preç...",None,50.0,1.0,11,1.0,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...,3700.0,R$ 950,R$ 4.650,R$ 2.200
1822,Aluguel,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,"{'rent': 4500, 'Condomínio': ' R$ 3.500 ', 'Pr...",None,100.0,NaN,3,3.0,[],None,None,None,None,"Sala Comercial e 3 banheiros para Alugar, 10...",4500.0,R$ 3.500,R$ 8.000,R$ 1.250
1823,Não informado,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,"{'rent': 3900, 'Condomínio': ' R$ 870 ', 'Preç...",None,87.0,NaN,1,2.0,[],None,None,None,None,"Sala Comercial e 1 banheiro para Alugar, 87 ...",3900.0,R$ 870,R$ 4.770,R$ 568
1824,Aluguel,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,"{'rent': 1970, 'Condomínio': ' R$ 720 ', 'Preç...",None,60.0,NaN,1,NaN,"[Ar-condicionado, Elevador, Condomínio fechado...",None,None,None,None,"Ponto Comercial e 1 banheiro para Alugar, 60...",1970.0,R$ 720,R$ 2.690,R$ 224


In [10]:
df.drop(columns=['total_price', 'description', 'transport', 'publisher_name', 'code'], inplace=True)

In [11]:
df['Condomínio'] = df['Condomínio'].str.replace('R$ ', '', regex=False)

In [12]:
df['Condomínio'] = df['Condomínio'].str.strip()

In [13]:
df['Condomínio'] = df['Condomínio'].replace({'Não informado': 0, 'Preço c/ condomínio':0, 'IPTU':'0'})

In [14]:
df['Condomínio'] = df['Condomínio'].str.replace('.', '', regex=False)

In [15]:
df['Condomínio'] = df['Condomínio'].fillna(0)

In [16]:
df['Condomínio'] = df['Condomínio'].astype(int)

In [17]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].replace({'R$ ':''}, regex=False).fillna(0)

In [18]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).fillna(0).astype(int)

In [19]:
df['IPTU'] = df['IPTU'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).replace({' Não informado ':0}, regex=False).fillna(0).astype(int)

In [20]:
df.drop(columns=['price'], inplace=True)

Endereço e amenities

Considerar os bairos.

In [21]:
df.shape

(1826, 13)

In [48]:
df.sample(frac=0.05)#pega 5% do dataframe

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU
1449,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0
844,"Vila Leonor, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,2.0,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",Apartamento com 2 Quartos e 2 banheiros para...,1850.0,780,2630,174
940,"Avenida Ramalho Ortigão, 356 - Vila Gumercindo...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,2.0,"[Piscina, Mobiliado, Elevador, Condomínio fech...",Apartamento com 2 Quartos e 2 banheiros para...,4500.0,920,5420,349
950,"Rua Kansas - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sobrado-3-q...,3.0,31,1.0,[],"Sobrado com 3 Quartos e 3 banheiros à Venda,...",2000000.0,0,0,0
547,"Rua das Fiandeiras - Vila Olímpia, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Piscina, Churrasqueira, Elevador, Varanda, Ac...",Apartamento com 1 Quarto e 1 banheiro para A...,2700.0,860,3560,170
...,...,...,...,...,...,...,...,...,...,...,...
474,"Rua Castrinópolis - Vila Santa Maria, São Paul...",https://www.vivareal.com.br/imovel/casa-1-quar...,1.0,1,NaN,[],"Casa com 1 Quarto e 1 banheiro para Alugar, ...",1000.0,0,0,0
489,"Rua Carneiro Leão, 385 - Brás, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,2.0,1,1.0,"[Churrasqueira, Elevador, Condomínio fechado, ...",Apartamento com 2 Quartos e 1 banheiro para ...,1280.0,370,1650,80
335,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0
324,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0


In [22]:
#organizar as amenities em ordem alfabética
df.amenities = df.amenities.apply(lambda x: sorted(x))

In [23]:
# identificando todas as amenities
amenities = []
df.amenities.apply(lambda x : amenities.extend(x))
amenities = set(sorted(amenities))

In [24]:
df_amenities = pd.DataFrame(df.amenities.apply(lambda x: {_:1 for _ in x}).to_list())

In [25]:
df_amenities.fillna(0, inplace=True)

In [27]:
pd.concat([df, df_amenities], axis=1).head(3)

,type,address,url,area,bedrooms,bathrooms,parking,amenities,title,rent,...,Drywall,Muro de vidro,Piso elevado,Poço artesiano,Muro e grade,Espaço teen,Pista de skate,Children care,Vista para lago,Centro de estética
0,Aluguel,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,80.0,2.0,21,1.0,"[Academia, Ar-condicionado, Churrasqueira, Con...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aluguel,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,40.0,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aluguel,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,38.0,1.0,1,1.0,"[Academia, Churrasqueira, Piscina]",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df

,type,address,url,area,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU
0,Aluguel,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,80.0,2.0,21,1.0,"[Academia, Ar-condicionado, Churrasqueira, Con...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,746,4646,225
1,Aluguel,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,40.0,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,0,0,0
2,Aluguel,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,38.0,1.0,1,1.0,"[Academia, Churrasqueira, Piscina]",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,726,3226,0
3,Não informado,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,27.0,1.0,11,NaN,"[Academia, Aceita animais, Bicicletário, Churr...",Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,350,2050,0
4,Aluguel,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,45.0,1.0,11,1.0,"[Academia, Aceita animais, Acesso para deficie...",Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,706,4206,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Aluguel,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,50.0,1.0,11,1.0,"[Academia, Aceita animais, Acesso para deficie...",Apartamento com 1 Quarto e 1 banheiro para A...,3700.0,950,4650,2200
1822,Aluguel,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,100.0,NaN,3,3.0,[],"Sala Comercial e 3 banheiros para Alugar, 10...",4500.0,3500,8000,1250
1823,Não informado,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,87.0,NaN,1,2.0,[],"Sala Comercial e 1 banheiro para Alugar, 87 ...",3900.0,870,4770,568
1824,Aluguel,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,60.0,NaN,1,NaN,"[Ar-condicionado, Condomínio fechado, Elevador...","Ponto Comercial e 1 banheiro para Alugar, 60...",1970.0,720,2690,224


In [29]:
df.to_excel('data/dados.xlsx', index=False)

## Identificar a geolocalização de cada endereço

In [4]:
df = pd.read_excel('data/dados.xlsx')

In [5]:
tqdm.pandas()

In [6]:
geolocator = Nominatim(user_agent="myApp")

In [44]:
g=geocode('Rua General Chagas Santos - Saúde, São Paulo - SP', addressdetails =True)
g

Location(Rua General Chagas Santos, Vila do Bosque, Saúde, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04138-030, Brasil, (-23.6157408, -46.6272546, 0.0))

In [8]:
g.raw

{'place_id': 281207922,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 943955517,
 'boundingbox': ['-23.6157949', '-23.6157211', '-46.6273066', '-46.6272303'],
 'lat': '-23.6157408',
 'lon': '-46.6272546',
 'display_name': 'Rua General Chagas Santos, Vila do Bosque, Saúde, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04138-030, Brasil',
 'class': 'highway',
 'type': 'tertiary',
 'importance': 0.8099999999999999,
 'address': {'road': 'Rua General Chagas Santos',
  'suburb': 'Vila do Bosque',
  'city_district': 'Saúde',
  'city': 'São Paulo',
  'municipality': 'Região Imediata de São Paulo',
  'county': 'Região Metropolitana de São Paulo',
  'state_district': 'Região Geográfica Intermediária de São Paulo',
  'state': 'São Paulo',
  'ISO3166-2-lvl4': 'BR-SP',
  'region': 'Região Sudeste',
  'postcode': '04138-030',
  

In [139]:
df['geocode'] = None

In [24]:
df = pd.read_excel('data/dados.xlsx')

In [9]:
geo_dic = {}

In [35]:
_geo.raw

{'place_id': 146009577,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 172343319,
 'boundingbox': ['-23.6270348', '-23.6246133', '-46.7364171', '-46.7349923'],
 'lat': '-23.6257265',
 'lon': '-46.7356239',
 'display_name': 'Rua Frei Duarte Jorge de Mendonça, Jardim Parque Morumbi, Vila Andrade, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 05640-000, Brasil',
 'class': 'highway',
 'type': 'tertiary',
 'importance': 1.1,
 'address': {'road': 'Rua Frei Duarte Jorge de Mendonça',
  'neighbourhood': 'Jardim Parque Morumbi',
  'suburb': 'Vila Andrade',
  'city': 'São Paulo',
  'municipality': 'Região Imediata de São Paulo',
  'county': 'Região Metropolitana de São Paulo',
  'state_district': 'Região Geográfica Intermediária de São Paulo',
  'state': 'São Paulo',
  'ISO3166-2-lvl4': 'BR-SP',
  'region': 'Região Sudeste',


In [12]:
%%time
for i, v in tqdm(df.iterrows(), total=df.shape[0]):
#     df.loc[i, 'geocode'] = 
    _geo = geocode(v['address'], addressdetails =True)
    if _geo is None:
        geo_dic[i] = False
    else:
        geo_dic[i] = {'suburb':_geo.raw.get('address', {}).get('suburb', False), 
                  'city_district':_geo.raw.get('address', {}).get('city_district', False), 
                  'address_class':_geo.raw.get('address', {}).get('address_class', False), 
                  'postcode':_geo.raw.get('address', {}).get('postcode', False)}

  2%|▊                                       | 37/1826 [01:07<53:15,  1.79s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rua Manuel Guedes - Itaim Bibi, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begin()
  File "c:\python\lib\http\client.py", line 307, in b

RateLimiter swallowed an error after 2 retries. Called with (*('Americanópolis, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begin()
  File "c:\python\lib\http\client.py", line 307, in begin
    version, statu

 15%|█████▉                                 | 278/1826 [09:26<46:07,  1.79s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Ipiranga, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begin()


RateLimiter caught an error, retrying (1/2 tries). Called with (*('Centro, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begin()
  File "c:\python\lib\http\client.py", line 307, in begin
    version, status, re

...

RateLimiter swallowed an error after 2 retries. Called with (*('Ipiranga, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begin()
  File "c:\python\lib\http\client.py", line 307, in begin
    version, status, rea

 78%|█████████████████████████████▊        | 1430/1826 [49:43<11:44,  1.78s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Avenida São Paulo Antigo - Real Parque, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in get

 86%|████████████████████████████████▌     | 1566/1826 [54:12<07:51,  1.81s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Cidade Monções, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.be

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Jardins, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begin()
  File "c:\python\lib\http\client.py", line 307, in begin
    version, status, r

Wall time: 1h 2min 40s


In [23]:
geo_dic_copy = geo_dic.copy()

In [24]:
for i, v in geo_dic.items():
    if v is False:
        geo_dic[i] = {'suburb': None,
      'city_district': None,
      'address_class': None,
      'postcode': None}

In [36]:
geo_dic

{0: {'suburb': False,
  'city_district': 'Santo Amaro',
  'address_class': False,
  'postcode': '04757-000'},
 1: {'suburb': False,
  'city_district': False,
  'address_class': False,
  'postcode': '04823-280'},
 2: {'suburb': False,
  'city_district': False,
  'address_class': False,
  'postcode': '16800-000'},
 3: {'suburb': 'Itaim Paulista',
  'city_district': False,
  'address_class': False,
  'postcode': '08541-310'},
 4: {'suburb': None,
  'city_district': None,
  'address_class': None,
  'postcode': None},
 5: {'suburb': 'Vila do Bosque',
  'city_district': 'Saúde',
  'address_class': False,
  'postcode': '04138-030'},
 6: {'suburb': 'Butantã',
  'city_district': 'Butantã',
  'address_class': False,
  'postcode': '05590-190'},
 7: {'suburb': 'Pinheiros',
  'city_district': 'Bairro de Pinheiros',
  'address_class': False,
  'postcode': '05423-030'},
 8: {'suburb': None,
  'city_district': None,
  'address_class': None,
  'postcode': None},
 9: {'suburb': 'Ipiranga',
  'city_distr

In [59]:
pd.DataFrame(geo_dic.values())

,suburb,city_district,address_class,postcode
0,False,Santo Amaro,False,04757-000
1,False,False,False,04823-280
2,False,False,False,16800-000
3,Itaim Paulista,False,False,08541-310
4,None,None,None,None
...,...,...,...,...
1821,None,None,None,None
1822,Itaim Bibi,False,False,01435-001
1823,Itaim Bibi,False,False,04564-004
1824,Bela Vista,Paulínia,False,13145-089


In [65]:
df = pd.concat([df, pd.DataFrame(geo_dic.values())], axis=1)

In [40]:
# df = pd.concat([df.drop(columns='geocode'), pd.DataFrame(geo_dic.items()).drop(columns=0).rename({1:'geocode'}, axis=1)], axis=1)

In [41]:
# %%time
# df['geocode'] = df['address'].progress_apply(geocode, addressdetails =True)#.progress_apply(lambda location: (location.latitude, location.longitude))
# df['coord'] = df['address'].progress_apply(geocode).progress_apply(lambda location: (location.latitude, location.longitude))
# df.head()

In [74]:
df.to_excel('data/dados.xlsx', index=False)

In [72]:
df.loc[0, 'address']

'Avenida Doutor Mário Vilas Boas Rodrigues, 175 - Santo Amaro, São Paulo - SP'

In [51]:
df = df[~df['address'].isna()]

In [13]:
_g = geocode('Mirandópolis, São Paulo - SP', addressdetails=True)
_g.raw

{'place_id': 297959194,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 298149,
 'boundingbox': ['-21.342', '-20.8305688', '-51.329', '-50.899'],
 'lat': '-21.1328313',
 'lon': '-51.1029422',
 'display_name': 'Mirandópolis, Região Imediata de Andradina, Região Geográfica Intermediária de Araçatuba, São Paulo, Região Sudeste, 16800-000, Brasil',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.7955028894050098,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png',
 'address': {'town': 'Mirandópolis',
  'municipality': 'Região Imediata de Andradina',
  'state_district': 'Região Geográfica Intermediária de Araçatuba',
  'state': 'São Paulo',
  'ISO3166-2-lvl4': 'BR-SP',
  'region': 'Região Sudeste',
  'postcode': '16800-000',
  'country': 'Brasil',
  'country_code': 'br'}}

In [4]:
df = pd.read_excel('data/dados.xlsx')

In [19]:
df#.loc[1, 'address']

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU,geocode,suburb,city_district,address_class,postcode
0,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,1.0,"['Academia', 'Ar-condicionado', 'Churrasqueira...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,746,4646,225,"Avenida Doutor Mário Vilas Boas Rodrigues, San...",NaN,Santo Amaro,0.0,04757-000
1,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,0,0,0,"Rua Fulgêncio Vertello, Cidade Dutra, São Paul...",Cidade Dutra,NaN,0.0,04823-280
2,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,1,1.0,"['Academia', 'Churrasqueira', 'Piscina']",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,726,3226,0,"Mirandópolis, Região Imediata de Andradina, Re...",NaN,NaN,0.0,16800-000
3,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,NaN,"['Academia', 'Aceita animais', 'Bicicletário',...",Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,350,2050,0,"Rua dos Estudantes, Itaim Paulista, São Paulo,...",Itaim Paulista,NaN,0.0,08541-310
4,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"['Academia', 'Aceita animais', 'Acesso para de...",Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,706,4206,296,NaN,Jardim Paulista,NaN,NaN,01421-000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"['Academia', 'Aceita animais', 'Acesso para de...",Apartamento com 1 Quarto e 1 banheiro para A...,3700.0,950,4650,2200,NaN,Santo Amaro,Santo Amaro,NaN,04713-001
1822,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,3,3.0,[],"Sala Comercial e 3 banheiros para Alugar, 10...",4500.0,3500,8000,1250,NaN,Itaim Bibi,NaN,0.0,01435-001
1823,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,1,2.0,[],"Sala Comercial e 1 banheiro para Alugar, 87 ...",3900.0,870,4770,568,NaN,Itaim Bibi,NaN,0.0,04564-004
1824,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,NaN,1,NaN,"['Ar-condicionado', 'Condomínio fechado', 'Ele...","Ponto Comercial e 1 banheiro para Alugar, 60...",1970.0,720,2690,224,NaN,Bela Vista,Paulínia,0.0,13145-089


In [7]:
not_found = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    if df.loc[i, ['suburb','city_district']].isna().all() and not pd.isna(df.loc[i, 'address']):#,'address_class','postcode'
        time.sleep(3)
        _geo = geocode(remove_bairro(df.loc[i, 'address']), addressdetails=True)
        if _geo is None:
            not_found.append(df.loc[i, 'address'])
#             console.print(f"[bold red]ID: {i}[/] Não encontrado o endereço [purple]{df.loc[i, 'address']}")
            continue
        suburb = _geo.raw.get('address', {}).get('suburb', False)
        if suburb is False:
            suburb = _geo.raw.get('address', {}).get('subdistrict', False)
        city_district = _geo.raw.get('address', {}).get('city_district', False)
        postcode = _geo.raw.get('address', {}).get('postcode', False)
        if suburb != False:
            df.loc[i, 'suburb'] = suburb
        if city_district != False:
            df.loc[i, 'city_district'] = city_district
        if postcode != False:
            df.loc[i, 'postcode'] = postcode

 13%|█████                                  | 238/1826 [00:39<04:32,  5.83it/s]

Rua Nhu-Guaçu - Nova Piraju, São Paulo - SP


 77%|█████████████████████████████▎        | 1409/1826 [04:40<02:46,  2.51it/s]

Rua Doutor Wladimir do Amaral, 101 - Jardim Peri-Peri, São Paulo - SP


100%|██████████████████████████████████████| 1826/1826 [05:40<00:00,  5.36it/s]


In [3]:
not_found = ['Alameda Itu, 78 - Jardins, São Paulo - SP',
 'Rua Carlos Facchina, 605 - Americanópolis, São Paulo - SP',
 'Avenida Professor Vicente Ráo - Jardim Petrópolis, São Paulo - SP',
 'Rua João Vitorino de Souza, 250 - Granja Julieta, São Paulo - SP',
 'Rua Intendência - Brás, São Paulo - SP',
 'Rua João Simões de Souza - Parque Rebouças, São Paulo - SP',
 'Rua Jacques Félix, 96 - Vila Nova Conceição, São Paulo - SP',
 'Rua Vitorino Carmilo, 217 - Barra Funda, São Paulo - SP',
 'Avenida João Carlos da Silva Borges - Vila Cruzeiro, São Paulo - SP',
 'Rua Barbacena, 1 - Vila Ema, São Paulo - SP',
 'Rua Sete de Outubro - Chácara Califórnia, São Paulo - SP',
 'Rua Passos, 249 - Belenzinho, São Paulo - SP',
 'Rua das Perobeiras - Chácara Santa Maria, São Paulo - SP',
 'Rua Groenlândia - Jardim América, São Paulo - SP',
 'Avenida Doutor Altino Arantes, 370 - Vila Clementino, São Paulo - SP',
 'Rua Soberana - Brooklin, São Paulo - SP',
 'Rua Hermes Ribeiro de Freitas, 328 - Jardim Elizabeth, São Paulo - SP',
 'Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',
 'Rua Capitão Manuel Novaes - Santana, São Paulo - SP',
 'Rua Alzira Maruyama, 264 - Vila Nova York, São Paulo - SP',
 'Rua Diogo Jácome, 518 - Vila Nova Conceição, São Paulo - SP',
 'Rua Maria Otília, 210 - Jardim Anália Franco, São Paulo - SP',
 'Avenida Onze de Junho - Vila Clementino, São Paulo - SP',
 'Rua Aristódemo Gazzotti, 633 - Vila das Belezas, São Paulo - SP',
 'Americanópolis, São Paulo - SP',
 'Rua Araporã - Vila Polopoli, São Paulo - SP',
 'Rua Bela Flor - Vila Mariana, São Paulo - SP',
 'Rua Cajuru - Belenzinho, São Paulo - SP',
 'Rua Alba - Vila Santa Catarina, São Paulo - SP',
 'Rua da Consolação, 2840 - Jardins, São Paulo - SP',
 'Rua Noronha Santos, 120 - Jardim Lucia, São Paulo - SP',
 'Rua da Paz, 2150 - Chácara Santo Antônio, São Paulo - SP',
 'Rua Pedro Pomponazzi - Chácara Klabin, São Paulo - SP',
 'Rua Tolstoi, 475 - Vila Brasilina, São Paulo - SP',
 'Rua Doutor Corinto Baldoino Costa, 42 - Tatuapé, São Paulo - SP',
 'Rua Belchior de Ordas, 200 - Vila Leonor, São Paulo - SP',
 'Rua Carlos Barbosa, 457 - Vila Sílvia, São Paulo - SP',
 'Rua Doutor Corinto Baldoino Costa, 42 - Tatuapé, São Paulo - SP',
 'Rua Professor Hasegawa, 914 - Itaquera, São Paulo - SP',
 'Estrada de Itapecerica - Vila das Belezas, São Paulo - SP',
 'Rua Zike Tuma, 116 - Interlagos, São Paulo - SP',
 'Rua Manoel Justiniano Quintão - Vila Palmeiras, São Paulo - SP',
 'Rua Haddock Lobo, 807 - Jardins, São Paulo - SP',
 'Rua Miguel Teles Júnior - Cambuci, São Paulo - SP',
 'Rua Martinho Prado, 43 - Bela Vista, São Paulo - SP',
 'Rua Lourenço de Almeida, 802 - Vila Nova Conceição, São Paulo - SP',
 'Alameda Joaquim Eugênio de Lima - Jardins, São Paulo - SP',
 'Rua Anhangüera, 725 - Barra Funda, São Paulo - SP',
 'Rua Antônio Pacheco Valente, 60 - Brooklin, São Paulo - SP',
 'Avenida Doutor Altino Arantes, 77 - Vila Clementino, São Paulo - SP',
 'Rua Nhu-Guaçu - Nova Piraju, São Paulo - SP',
 'Rua Baquirivu - Cidade Ademar, São Paulo - SP',
 'Alameda Eduardo Prado - Campos Eliseos, São Paulo - SP',
 'Rua Atucuri, 637 - Tatuapé, São Paulo - SP',
 'Rua Antônio das Chagas - Chácara Santo Antônio, São Paulo - SP',
 'Rua Brás Cardoso, 654 - Vila Nova Conceição, São Paulo - SP',
 'Rua Pascoal Vita, 405 - Vila Madalena, São Paulo - SP',
 'Rua Helvétia, 661 - Campos Eliseos, São Paulo - SP',
 'Rua das Perobas, 435 - Vila Parque Jabaquara, São Paulo - SP',
 'Avenida Professor Osvaldo de Oliveira - Jardim Helena, São Paulo - SP',
 'Avenida Doutor Mário Vilas Boas Rodrigues, 161 - Jardim Dom Bosco, São Paulo - SP',
 'Ipiranga, São Paulo - SP',
 'Rua Haddock Lobo - Jardim América, São Paulo - SP',
 'Rua Doutor Tomás Alves, 172 - Vila Mariana, São Paulo - SP',
 'Centro, São Paulo - SP',
 'Centro, São Paulo - SP',
 'Rua Joinville, 55 - Vila Mariana, São Paulo - SP',
 'Rua Luís Pacheco, 287 - Ponte Pequena, São Paulo - SP',
 'Alameda Jaú, 796 - Jardins, São Paulo - SP',
 'Avenida Nova Independência, 37 - Brooklin, São Paulo - SP',
 'Rua Doutor Penaforte Mendes, 157 - Bela Vista, São Paulo - SP',
 'Rua Satuba - Vila Pauliceia, São Paulo - SP',
 'Rua Luiz dos Santos Cabral, 197 - Jardim Anália Franco, São Paulo - SP',
 'Avenida Padres Olivetanos - Vila Matilde, São Paulo - SP',
 'Rua Ministro Luiz Gallotti, 74 - Cidade Monções, São Paulo - SP',
 'Rua dos Pargos - Vila Guacuri, São Paulo - SP',
 "Rua Gabriele D'Annunzio, 624 - Campo Belo, São Paulo - SP",
 'Rua Professor Filadelfo Azevedo, 50 - Vila Nova Conceição, São Paulo - SP',
 'Rua Deputado Laércio Corte, 1333 - Panamby, São Paulo - SP',
 'Avenida Senador Casimiro da Rocha - Mirandópolis, São Paulo - SP',
 'Rua Lorenzo Valla - Chácara Klabin, São Paulo - SP',
 'Rua Pedrina Maria da Silva Valente - Parque Munhoz, São Paulo - SP',
 'Rua Riacho da Cruz, 446 - Parque Císper, São Paulo - SP',
 'Rua Manuel de Paiva, 76 - Vila Mariana, São Paulo - SP',
 "Rua Gabriele D'Annunzio, 296 - Campo Belo, São Paulo - SP",
 'Avenida Nossa Senhora do Sabará, 100 - Vila Sofia, São Paulo - SP',
 'Rua Acruás, 88 - Vila Congonhas, São Paulo - SP',
 'Rua Doutor Chibata Miyakoshi - Panamby, São Paulo - SP',
 'Rua Januário Zingaro - Jardim Paris, São Paulo - SP',
 'Vila Paranaguá, São Paulo - SP',
 'Rua Jaceru, 376 - Vila Gertrudes, São Paulo - SP',
 'Rua Diogo Jácome, 554 - Vila Nova Conceição, São Paulo - SP',
 'Rua Doutor Penaforte Mendes, 140 - Bela Vista, São Paulo - SP',
 'Rua Barão do Triunfo - Brooklin, São Paulo - SP',
 'Rua Doutor Cesário Mota Júnior - Vila Buarque, São Paulo - SP',
 'Rua Barbosa Lopes - Granja Julieta, São Paulo - SP',
 'Rua Padre Machado, 806 - Vila Clementino, São Paulo - SP',
 'Rua Ester Samara, 111 - Jardim Claudia, São Paulo - SP',
 'Rua Haddock Lobo, 1175 - Jardim América, São Paulo - SP',
 'Avenida Senador Casimiro da Rocha, 1079 - Mirandópolis, São Paulo - SP',
 'Rua Castrinópolis - Vila Santa Maria, São Paulo - SP',
 'Rua Bueno Brandão, 403 - Vila Nova Conceição, São Paulo - SP',
 'Jardim Marpu, São Paulo - SP',
 'Avenida João Batista Santiago, 282 - Guaianases, São Paulo - SP',
 'Rua Luís Correia de Melo, 306 - Granja Julieta, São Paulo - SP',
 'Rua Genebra, 230 - Bela Vista, São Paulo - SP',
 'Rua Bela Cintra - Jardim América, São Paulo - SP',
 'Rua Santa Madalena, 274 - Liberdade, São Paulo - SP',
 'Rua Filipa Moniz Perestrello, 576 - Vila Fachini, São Paulo - SP',
 'Rua Santa Madalena, 290 - Liberdade, São Paulo - SP',
 'Avenida Nhandu - Planalto Paulista, São Paulo - SP',
 "Rua Gabriele D'Annunzio, 48 - Campo Belo, São Paulo - SP",
 'Rua Augusta, 2312 - Jardins, São Paulo - SP',
 'Rua Aguiar de Barros - Bela Vista, São Paulo - SP',
 'Rua Jaguaribe, 252 - Vila Buarque, São Paulo - SP',
 'Rua Marquês de Valença - Alto da Mooca, São Paulo - SP',
 'Rua Alexandre Dumas, 1392 - Chácara Santo Antônio, São Paulo - SP',
 'Rua das Fiandeiras - Vila Olímpia, São Paulo - SP',
 'Avenida Engenheiro Luiz Carlos Berrini - Brooklin, São Paulo - SP',
 'Rua Jaguaribe - Vila Buarque, São Paulo - SP',
 'Alameda das Boninas - Vila Mariana, São Paulo - SP',
 'Rua São João da Bocaina, 62 - Chácara Mafalda, São Paulo - SP',
 'Rua Abílio Soares, 1251 - Ibirapuera, São Paulo - SP',
 'Jardins, São Paulo - SP',
 'Rua Doutor Luiz Migliano, 923 - Morumbi, São Paulo - SP',
 'Rua Comendador Francisco Pettinati - Jardim Monte Kemel, São Paulo - SP',
 'Rua Marcos Lopes, 272 - Ibirapuera, São Paulo - SP',
 'Rua Professor Augusto Monjardino - Butantã, São Paulo - SP',
 'Rua Lamartine Babo, 0 - Chácara Santo Antônio, São Paulo - SP',
 'Rua Erva Grossa - Lajeado, São Paulo - SP',
 'Rua Manuel de Paiva - Vila Mariana, São Paulo - SP',
 'Rua Barão de Capanema - Jardins, São Paulo - SP',
 'Rua Gabriel de Gale - Monte Azul, São Paulo - SP',
 'Rua David Romero Pare, 35 - Morumbi, São Paulo - SP',
 'Rua Antônio Olinto, 189 - Jardim Oriental, São Paulo - SP',
 'Travessa Augusto Lovato, 0 - Vila Conde do Pinhal, São Paulo - SP',
 'Rua Padre Machado, 455 - Bosque da Saúde, São Paulo - SP',
 'Rua Simões Pinto, 138 - Parque Jabaquara, São Paulo - SP',
 'Rua Quinze de Setembro, 174 - Saúde, São Paulo - SP',
 'Rua Doutor Carlos Aldrovandi, 85 - Panamby, São Paulo - SP',
 'Vila Uberabinha, São Paulo - SP',
 'Rua Estevão Baião, 520 - Vila Congonhas, São Paulo - SP',
 'Rua Flórida, 1950 - Cidade Monções, São Paulo - SP',
 'Rua Olavo Bilac, 242 - Vila Sofia, São Paulo - SP',
 'Rua Jacques Félix - Vila Nova Conceição, São Paulo - SP',
 'Jardim Bom Refúgio, São Paulo - SP',
 'Rua da Paz, 2150 - Chácara Santo Antônio, São Paulo - SP',
 'Avenida Onze de Junho, 1105 - Vila Clementino, São Paulo - SP',
 'Rua Porto Alegre - Móoca, São Paulo - SP',
 'Rua Henri Dunant, 792 - Chácara Santo Antônio, São Paulo - SP',
 'Alameda Lorena, 1852 - Jardins, São Paulo - SP',
 'Rua Leopoldo de Bulhões, 59 - Vila Clementino, São Paulo - SP',
 'Avenida Ministro Petrônio Portela, 1901 - Vila Amélia, São Paulo - SP',
 'Rua Atucuri, 637 - Tatuapé, São Paulo - SP',
 'Rua Abílio Soares, 1251 - Ibirapuera, São Paulo - SP',
 'Avenida Doutor Guilherme Dumont Vilares - Jardim Londrina, São Paulo - SP',
 'Rua Rafael Ficondo - Vila Brasilina, São Paulo - SP',
 'Rua Osiris Magalhães de Almeida - Jardim Monte Kemel, São Paulo - SP',
 "Rua Gabriele D'Annunzio, 390 - Campo Belo, São Paulo - SP",
 'Rua Celso Ramos - Vila Andrade, São Paulo - SP',
 'Avenida Casa Grande, 900 - Vila Ema, São Paulo - SP',
 'Rua Barão do Triunfo, 314 - Brooklin, São Paulo - SP',
 'Jardim Leônidas Moreira I, São Paulo - SP',
 'Rua Doutor Odilon - Vila Brasilina, São Paulo - SP',
 'Avenida Padre Arlindo Vieira, 700 - Vila Vermelha, São Paulo - SP',
 'Rua Manoel de Sousa Azevedo, 33 - Freguesia do Ó, São Paulo - SP',
 'Alameda Tietê - Jardins, São Paulo - SP',
 'Rua Joaquim Ferreira, 124 - Água Branca, São Paulo - SP',
 'Rua Uhland - Vila Ema, São Paulo - SP',
 'Rua Brigadeiro Galvão, 674 - Barra Funda, São Paulo - SP',
 'Rua Dom Antônio Galvão - Vila Gumercindo, São Paulo - SP',
 'Alameda Gabriel Monteiro da Silva - Jardim América, São Paulo - SP',
 'Rua Diego de Castilho, 150 - Morumbi, São Paulo - SP',
 'Avenida Yervant Kissajikian - Jardim Marajoara, São Paulo - SP',
 'Rua Capão da Serra - Vila Brasílio Machado, São Paulo - SP',
 'Rua Francisco Gouveia, 26 - Quarta Parada, São Paulo - SP',
 'Rua Evolução - Vila Brasilina, São Paulo - SP',
 'Rua Anhangüera, 289 - Barra Funda, São Paulo - SP',
 'Alameda dos Tacaúnas, 547 - Planalto Paulista, São Paulo - SP',
 'Rua França Carvalho - Alto da Mooca, São Paulo - SP',
 'Rua Trajano Reis, 47 - Jardim das Vertentes, São Paulo - SP',
 'Rua Torres da Barra, 275 - Água Branca, São Paulo - SP',
 'Rua Oscar Caravelas, 280 - Vila Madalena, São Paulo - SP',
 'Rua Iepê - Vila Anhanguera, São Paulo - SP',
 'Rua Savigni, 200 - Vila Bela, São Paulo - SP',
 'Rua Mateus Garcia, 919 - Vila Irmãos Arnoni, São Paulo - SP',
 'Rua Dom Paulo Pedrosa, 720 - Real Parque, São Paulo - SP',
 'Rua Charim, 47 - Vila Carmosina, São Paulo - SP',
 'Rua Jorge da Silva Moreno - Jardim Cliper, São Paulo - SP',
 'Avenida José Maria da Silva Paranhos, 105 - Jardim Monte Kemel, São Paulo - SP',
 'Avenida Nossa Senhora do Sabará - Vila Sofia, São Paulo - SP',
 'Rua Rubens Meireles - Barra Funda, São Paulo - SP',
 'Jardins, São Paulo - SP',
 'Rua Padre Lebret, 725 - Jardim Leonor, São Paulo - SP',
 'Avenida Ordem e Progresso, 1084 - Jardim das Laranjeiras, São Paulo - SP',
 'Rua Bernardo Tavares - Vila Pirajussara, São Paulo - SP',
 'Rua Serra de São Domingos, 99 - Vila Carmosina, São Paulo - SP',
 'Rua República do Iraque - Brooklin, São Paulo - SP',
 'Rua Guaraiúva, 531 - Cidade Monções, São Paulo - SP',
 'Rua Martiniano de Carvalho, 807 - Paraíso, São Paulo - SP',
 'Rua Doutor Martins de Oliveira, 312 - Jardim Londrina, São Paulo - SP',
 'Rua Francisco Luiz de Souza Júnior, 398 - Água Branca, São Paulo - SP',
 'Rua Cotinga, 236 - Vila Nova Curuçá, São Paulo - SP',
 'Avenida Doutor Luís Arrobas Martins, 114 - Capela do Socorro, São Paulo - SP',
 'Rua Álvaro de Carvalho, 48 - Consolação, São Paulo - SP',
 'Rua Sansão Alves dos Santos - Cidade Monções, São Paulo - SP',
 'Rua Barão do Triunfo, 478 - Brooklin, São Paulo - SP',
 'Rua General Irulegui Cunha, 180 - Jardim  Independencia, São Paulo - SP',
 'Rua Oscar Freire - Jardim América, São Paulo - SP',
 'Rua dos Cafezais - Jardim Prudência, São Paulo - SP',
 'Rua José Carlos de Toledo Piza, 175 - Jardim Parque Morumbi, São Paulo - SP',
 'Jardins, São Paulo - SP',
 'Rua Oscar José Alves, 54 - Jardim Carombe, São Paulo - SP',
 'Rua João Monteiro, 70 - Campos Eliseos, São Paulo - SP',
 'Rua Tuim - Vila Uberabinha, São Paulo - SP',
 'Rua Tenente Nilton Marcilio - Vila Antonina, São Paulo - SP',
 'Rua Professor José Horácio Meirelles Teixeira - Morumbi, São Paulo - SP',
 'Rua Ministro Luiz Gallotti, 74 - Vila Cordeiro, São Paulo - SP',
 'Avenida do Cursino, 5780 - Vila Moraes, São Paulo - SP',
 'Rua Maria Otília, 210 - Jardim Anália Franco, São Paulo - SP',
 'Rua Coronel Diogo, 594 - Aclimação, São Paulo - SP',
 'Rua Luís Correia de Melo, 250 - Vila Cruzeiro, São Paulo - SP',
 'Rua Manuel Leiroz - Vila Penteado, São Paulo - SP',
 'Rua Adolfo Gordo, 100 - Campos Eliseos, São Paulo - SP',
 'Rua Coronel Fawcett - Vila Moraes, São Paulo - SP',
 'Rua Said Aiach, 277 - Ibirapuera, São Paulo - SP',
 'Parque Boturussu, São Paulo - SP',
 'Rua Laurentino Xavier dos Santos, 421 - Artur Alvim, São Paulo - SP',
 'Rua Antônio Carlos Pereira - Parque São Domingos, São Paulo - SP',
 'Rua Adele, 119 - Chácara Santo Antônio, São Paulo - SP',
 "Avenida Sargento Geraldo Sant'Ana, 301 - Jardim Taquaral, São Paulo - SP",
 'Alameda Ministro Rocha Azevedo, 482 - Jardins, São Paulo - SP',
 'Vila Cosmopolita, São Paulo - SP',
 'Rua Teodoro de Beaurepaire, 208 - Vila Dom Pedro I, São Paulo - SP',
 'Rua dos Jatobás, 80 - Vila Parque Jabaquara, São Paulo - SP',
 'Rua Ourânia - Vila Madalena, São Paulo - SP',
 'Rua Manoel de Sousa Azevedo, 266 - Sítio Morro Grande, São Paulo - SP',
 'Rua Verbo Divino, 1323 - Chácara Santo Antônio, São Paulo - SP',
 'Rua Júlio Kazakevicius - Jardim Represa, São Paulo - SP',
 'Rua Melo Alves, 392 - Jardins, São Paulo - SP',
 'Rua Leopoldo de Bulhões, 59 - Vila Clementino, São Paulo - SP',
 'Avenida São Paulo Antigo - Real Parque, São Paulo - SP',
 'Rua Oscar José Alves, 54 - Jardim Carombe, São Paulo - SP',
 'Rua Flamengo - Tatuapé, São Paulo - SP',
 'Rua David Eid - Jardim Consórcio, São Paulo - SP',
 'Rua David Eid, 1907 - Interlagos, São Paulo - SP',
 'Rua Toledo Barbosa, 290 - Belenzinho, São Paulo - SP',
 'Rua das Perobeiras - Chácara Santa Maria, São Paulo - SP',
 'Rua General Bráulio Guimarães, 208 - Jardim Promissão, São Paulo - SP',
 'Rua Pedro Pomponazzi - Chácara Klabin, São Paulo - SP',
 'Rua Olímpia de Almeida Prado, 27 - Barra Funda, São Paulo - SP',
 'Rua Deputado João Sussumu Hirata, 6675 - Jardim Sul, São Paulo - SP',
 'Rua Roque Giangrande Filho - Jardim Taquaral, São Paulo - SP',
 'Alameda Jaú, 1477 - Jardins, São Paulo - SP',
 'Alameda Itu - Jardins, São Paulo - SP',
 'Rua Laguna - Chácara Santo Antônio, São Paulo - SP',
 'Rua Jaci - Vila Mariana, São Paulo - SP',
 'Rua Martinho Claro, 66 - Conjunto Residencial Butantã, São Paulo - SP',
 'Rua Savério Quadrio, 0 - Butantã, São Paulo - SP',
 'Rua Carolina Fonseca, 407 - Vila Santana, São Paulo - SP',
 'Avenida Onze de Junho, 1105 - Vila Clementino, São Paulo - SP',
 'Rua Nova Jerusalém - Chácara Santo Antônio Zona Leste, São Paulo - SP',
 'Avenida Conceição, 649 - Vila Paiva, São Paulo - SP',
 'Avenida Duque de Caxias - Campos Eliseos, São Paulo - SP',
 'Rua Augusta, 2312 - Jardins, São Paulo - SP',
 'Rua Henri Dunant - Chácara Santo Antônio, São Paulo - SP',
 'Rua Vapabussu, 66 - Jardim Aeroporto, São Paulo - SP',
 'Rua Osiris de Camargo, 100 - Parque Munhoz, São Paulo - SP',
 'Rua Guaperoba - Vila Brasílio Machado, São Paulo - SP',
 'Rua David Eid, 1907 - Interlagos, São Paulo - SP',
 'Jardins, São Paulo - SP',
 'Rua Diego de Castilho, 150 - Panamby, São Paulo - SP',
 'Rua Arcipreste Andrade - Ipiranga, São Paulo - SP',
 'Rua Carlos Maria Steimberg, 296 - Vila Ré, São Paulo - SP',
 'Rua Pirineus, 96 - Campos Eliseos, São Paulo - SP',
 'Avenida Doutor Mário Vilas Boas Rodrigues, 175 - Chácara Santo Antônio, São Paulo - SP',
 'Rua Francisco Rebelo - Vila California, São Paulo - SP',
 'Rua Octacílio de Carvalho Lopes, 0 - Jardim São Luís, São Paulo - SP',
 'Avenida Yervant Kissajikian - Jardim Marajoara, São Paulo - SP',
 'Avenida Doutor Altino Arantes, 77 - Vila Clementino, São Paulo - SP',
 'Vila Rui Barbosa, São Paulo - SP',
 'Alameda Jaú, 135 - Consolação, São Paulo - SP',
 'Avenida Doutor Altino Arantes, 245 - Vila Clementino, São Paulo - SP',
 'Rua Acarapereira, 50 - Sítio Morro Grande, São Paulo - SP',
 'Rua Amoipira, 201 - Vila Isa, São Paulo - SP',
 'Rua Pedro da Rocha Ferreira - Vila Rubi, São Paulo - SP',
 'Rua Cabo Estácio da Conceição - Parque Maria Helena, São Paulo - SP',
 'Rua Doutor Mário Ferraz - Jardim Paulistano, São Paulo - SP',
 'Rua Carlos Facchina, 605 - Americanópolis, São Paulo - SP',
 'Rua Doutor Clemente Jobim, 65 - Aclimação, São Paulo - SP',
 'Rua Professor Benedito Alarico de Castro Borelli - Parque São Domingos, São Paulo - SP',
 'Rua Coronel Bento Pires, 40 - Móoca, São Paulo - SP',
 'Rua Ana Vieira de Carvalho - Jardim Panorama, São Paulo - SP',
 'Rua Carlos Gonzalez, 94 - Vila Curuçá, São Paulo - SP',
 'Jardins, São Paulo - SP',
 'Rua Siqueira Bueno, 0 - Belenzinho, São Paulo - SP']

In [10]:
df.to_excel('data/dados.xlsx', index=False)

# load description

In [2]:
df = pd.read_excel('data/dados.xlsx')

In [36]:
ndf = pd.DataFrame(columns=['title', 'desc'])

In [22]:
ndf.loc[-1] = [2, 3] 

In [23]:
ndf

,title,desc
-1,2,3


In [24]:
ndf.index + 1

Int64Index([0], dtype='int64')

In [39]:
scrap = Request()
x = 0
#Try load description
for i, v in tqdm(df.iterrows(), total=len(df)):
    obj = scrap.get_info(v.url)
    if obj.code != 'INATIVO':
        ndf.loc[v.url] = [obj.title.strip(), obj.description.strip()]

100%|████████████████████████████████████| 1826/1826 [1:29:47<00:00,  2.95s/it]


In [50]:
df = pd.concat([df, ndf], axis=0)

In [51]:
df.to_excel('data/dados.xlsx', index=False)